In [ ]:
%load_ext autoreload
%autoreload 2
import os

from mair import notebook_utils
from mair import papers_processing_utils
import spacy 
from tqdm import tqdm
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import gensim
from spacy import displacy
from collections import Counter

tqdm.pandas()

In [ ]:
INPUT_FOLDER = "../data/arxiv_dump/cleaned_texts/"
NOUNS = ['fairness', 'explanation', 'bias']

input_files_pattern = os.path.join(INPUT_FOLDER, "*.txt")

In [ ]:
df = notebook_utils.load_texts_as_df(input_files_pattern)
df

In [ ]:
en=spacy.load('en_core_web_lg')
#en=spacy.load('en_core_web_sm')

docs = df.text[:20].progress_apply(en)

## Most common lemmas

In [ ]:
lemmas = docs.progress_apply(
    papers_processing_utils.get_lemmas)
all_lemas = []
for l in lemmas:
    all_lemas+=l
    
unigrams = nltk.FreqDist(all_lemas)
plt.figure(figsize=(10,6))

unigrams.plot(30)

## Keywords

In [ ]:
keywords = lemmas.progress_apply(" ".join).apply(gensim.summarization.keywords).str.split('\n')
all_keywords = []
for k in keywords:
    all_keywords+=k
plt.figure(figsize=(10,6))
nltk.FreqDist(all_keywords).plot(30)

# Nouns analysis

In [ ]:
noun = NOUNS[0]

## Sentences with noun

In [ ]:
found_sents = []
for doc in docs:
    for sent in doc.sents:
        for token in sent:
            if token.lemma_ == noun:
                found_sents.append(sent)
                break
                
len(found_sents)

In [ ]:
found_sents

In [ ]:
lemmas = list(map(papers_processing_utils.get_lemmas, found_sents))
sents_all_words = []
for l in lemmas:
    sents_all_words+=l

### Words in sentences with noun

In [ ]:
plt.figure(figsize=(10,6))
nltk.FreqDist(sents_all_words).plot(30)

## Noun chunks with noun 

Noun and descriptions

In [ ]:
found_chunks = []
for doc in docs:
    for noun_chunk in doc.noun_chunks:
        for token in noun_chunk:
            if token.lemma_ == noun:
                found_chunks.append(noun_chunk)
                break
len(found_chunks)

In [ ]:
entities = [
    ent
    for doc in docs
    for ent in doc.ents
    if ent.label_ not in {"CARDINAL", "DATE", "TIME", "ORDINAL", "MONEY", "PERCENT"}
]
found_chunks

## Noun heads analysis

What is noun describing

In [ ]:
heads = []

for sent in found_sents:
    for token in sent: 
        if token.lemma_ == noun:
            head = token.head
            heads.append(head)

In [ ]:
heads

In [ ]:
displacy.render(sent, style='dep')

# NER analysis

In [ ]:
entities = [
    ent
    for doc in docs
    for ent in doc.ents
    if ent.label_ not in {"CARDINAL", "DATE", "TIME", "ORDINAL", "MONEY", "PERCENT"}
]

labels = Counter([ent.label_ for ent in entities])
labels

In [ ]:
#entities

## Organisations

In [ ]:
[ent for ent in entities if ent.label_=='ORG']